# Post Calibration
In this we look at the quality of the calibrated impedance functions.
---
1. Recalculate shortest path results using the calibrated coefficients
1. Calculate objective functions and other performance metrics
1. Get route attribute summary for the chosen/shortest/modeled routes (right now)


Still Working On
1. Look at where calibrated function did the best/worst job for both the training/testing set
1. Cluster/segment results based on loss function value?
4. Export for application in BikewaySim

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
from shapely.ops import LineString, MultiLineString
from importlib import reload

from bikewaysim.paths import config
from bikewaysim.network import modeling_turns
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor

from step_1_calibration_experiments import all_calibrations, full_model


Run post calibration for all results

In [ ]:
#BUG instead of using the calibration network for these processes, use the full network
stochastic_optimization.post_calibration_loss(False) # for general

In [ ]:
stochastic_optimization.post_calibration_loss(True) # for user specific results

In [ ]:
reload(stochastic_optimization)
beta_vals = stochastic_optimization.post_calibration_betas(True)
full_model_cols = [item['col'] for item in full_model]
beta_vals = beta_vals[beta_vals[full_model_cols].notna().all(axis=1)].dropna(axis=1)
beta_vals

In [ ]:
# disaggregate only really makes sense for all trips calibrations or testing
# reload(stochastic_optimization)
# disaggregated_loss = stochastic_optimization.post_calibration_disaggregate(True)
# disaggregated_loss

In [ ]:
reload(stochastic_optimization)
aggregated_loss = stochastic_optimization.post_calibration_aggregated(True)
aggregated_loss

In [ ]:
# add
merged = pd.merge(beta_vals,aggregated_loss,on=['userid','run_number','calibration'])
merged

In [ ]:
# which calibration method generally resulted in the best outcomes?
merged.groupby('calibration')['jaccard_exact_mean','jaccard_exact_total','jaccard_buffer_mean','jaccard_buffer_total'].mean()

In [ ]:
stochastic_optimization.shortest_loss()

In [ ]:
with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('rb') as fh:
    full_set = pickle.load(fh)

In [ ]:
for x in ['origin_node','destination_node','trip_start_time','match_ratio','matched_edges','shortest_edges']:
    [item.pop(x) for tripid, item in full_set.items()]

In [ ]:
pd.DataFrame.from_dict(full_set,orient='index')

In [ ]:
reload(stochastic_optimization)
stochastic_optimization.shortest_aggregated()

In [ ]:
user_shortest = stochastic_optimization.shortest_aggregated(True)
user_shortest.set_index('userid',inplace=True)

In [ ]:
user_shortest

In [ ]:
merged.groupby('userid')['jaccard_exact_mean','jaccard_exact_total','jaccard_buffer_mean','jaccard_buffer_total'].max()

In [ ]:
merged.groupby(['userid','calibration']).std().dropna()

In [ ]:
userid = '48_gdot (1)'.split('_',maxsplit=1)[0]
calibration_name = '48_gdot (1)'.split('_',maxsplit=1)[1].split('(')[0]
run_number = '48_gdot (1)'.split('_',maxsplit=1)[1]
test = '48_gdot'


In [ ]:
import re
pattern = r'\((\d+)\)'
re.findall(pattern,test)

In [ ]:
aggregated_loss

In [ ]:
aggregated_loss = stochastic_optimization.post_calibration_aggregated()
beta_vals = stochastic_optimization.post_calibration_betas()

In [ ]:
aggregated_loss

In [ ]:
disaggregated_loss = stochastic_optimization.post_calibration_disaggregate(True)
disaggregated_loss

In [ ]:
beta_vals

In [ ]:
beta_vals

# Import Relevant Files

In [ ]:
# links, turns, length_dict, geo_dict, turn_G = stochastic_optimization.import_calibration_network(config)
# with (config['calibration_fp']/'ready_for_calibration.pkl').open('rb') as fh:
#     full_set = pickle.load(fh)
# full_ods = stochastic_optimization.match_results_to_ods_w_year(full_set)
# Retreive shortest and chosen stats first (already calculated the shortest paths)
# for key, item in full_set.items():
#     # extract chosen and shortest routes
#     chosen = item['matched_edges'].values
#     shortest = item['shortest_edges'].values

#     #compute the loss values (store the intermediates too so total vs mean methods can be compared)
#     shortest_jaccard_exact_intersection, shortest_jaccard_exact_union =  stochastic_optimization.jaccard_exact(chosen,shortest,length_dict)
#     shortest_jaccard_exact = shortest_jaccard_exact_intersection / shortest_jaccard_exact_union
#     shortest_jaccard_buffer_intersection, shortest_jaccard_buffer_union =  stochastic_optimization.jaccard_buffer(chosen,shortest,geo_dict)
#     shortest_jaccard_buffer = shortest_jaccard_buffer_intersection / shortest_jaccard_buffer_union

#     # add to full set in additon to what's already there
#     full_set[key].update({
#         'chosen_length': round(np.array([length_dict.get(tripid[0],False) for tripid in chosen]).sum()/5280,2),
#         'shortest_length': round(np.array([length_dict.get(tripid[0],False) for tripid in shortest]).sum()/5280,2),
#         'chosen_detour': round(stochastic_optimization.detour_factor(chosen,shortest,length_dict),2),
#         'shortest_jaccard_exact': round(shortest_jaccard_exact,2),
#         'shortest_jaccard_exact_intersection': round(shortest_jaccard_exact_intersection,2),
#         'shortest_jaccard_exact_union': round(shortest_jaccard_exact_union,2),
#         'shortest_jaccard_buffer': round(shortest_jaccard_buffer,2),
#         'shortest_jaccard_buffer_intersection': round(shortest_jaccard_buffer_intersection,2),
#         'shortest_jaccard_buffer_union': round(shortest_jaccard_buffer_union,2),
#     })

# # export new version
# with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('wb') as fh:
#     pickle.dump(full_set,fh)


# Post process calibration results

In [ ]:
# # get all of the current calibration results
# calibration_result_fps = list((config['calibration_fp']/"calibration_results").glob('*.pkl'))

# # pick a specific model
# # specific_model = 'jaccard_'
# # calibration_result_fps = [x for x in calibration_result_fps if specific_model in x.stem.split('(')[0].strip()]

# Get dataframe of calibrated betas (fast)
# beta_vals = {}
# for idx, calibration_result_fp in enumerate(calibration_result_fps):
#     with calibration_result_fp.open('rb') as fh:
#         calibration_result = pickle.load(fh)
#     beta_vals[calibration_result_fp.stem] = {x['col']:x['beta'] for x in calibration_result['betas_tup']}
# beta_vals = pd.DataFrame().from_dict(beta_vals,orient='index')
# # beta_vals.std()
# beta_vals.round(2)

With the betas, calculate all of the shortest paths

In [ ]:
# beta_vals = {}

# from importlib import reload
# reload(stochastic_optimization)

# #NOTE TEMP
# # calibration_result_fps = [calibration_result_fps[0]]
# set_to_zero = []
# set_to_inf = []
# # full_ods = full_ods[0:10]

# modeled_paths_dicts = {}
# for calibration_result_fp in tqdm(calibration_result_fps):
    
#     with calibration_result_fp.open('rb') as fh:
#         calibration_result = pickle.load(fh)

#     beta_vals[calibration_result_fp.stem] = {x['col']:x['beta'] for x in calibration_result['betas_tup']}

#     base_impedance_col = "travel_time_min" # set the base impedance (default is travel time)
#     betas = [x['beta'] for x in calibration_result['betas_tup']] # get betas

#     # create a copy of links so we don't mutate it
#     links0 = links.copy()
    
#     # reset the link costs
#     stochastic_optimization.back_to_base_impedance(base_impedance_col,links0,turns,turn_G)

#     # do initial impedance update
#     # if infra is off street (i.e., the link should no longer be traversable)
#     links0['link_cost_override'] = False
#     stochastic_optimization.impedance_update(betas,calibration_result['betas_tup'],
#                         stochastic_optimization.link_impedance_function,
#                         base_impedance_col,
#                         stochastic_optimization.turn_impedance_function,
#                         links0,turns,turn_G)

#     trips_years = set()
#     results_dict = {}
#     for start_node, end_node, year in full_ods:
#         if year not in trips_years:
#             # print('Remaking network for',year)
#             trips_years.add(year) # add it to the years already looked at
#             if (links0['year'] > year).any(): # only then should we run this
#                 # print('Re-making network to year',year)
                
#                 # if infra is on street (i.e., the link is still traversable but the impedance doesn't apply)
#                 links0.loc[links0['year']>year,set_to_zero] = 0 
#                 links0.loc[(links0['year']>year) & (links0.loc[:,set_to_inf]==1).any(axis=1),'link_cost_override'] = True
                
#                 # re-run network update
#                 updated_edge_costs = stochastic_optimization.impedance_update(betas,calibration_result['betas_tup'],
#                         stochastic_optimization.link_impedance_function,
#                         base_impedance_col,
#                         stochastic_optimization.turn_impedance_function,
#                         links0,turns,turn_G)
            
#         results_dict[(start_node,end_node)] = stochastic_optimization.impedance_path(turns,turn_G,links0,start_node,end_node)

#     # before this, retrieve the tripid?

#     # export
#     with (config['calibration_fp']/'post_calibration_routing'/(calibration_result_fp.stem+'.pkl')).open('wb') as fh:
#         pickle.dump(results_dict,fh)


# Calculate loss and objective functions

In [ ]:
# post_calibration_routing_fps = list((config['calibration_fp']/"post_calibration_routing").glob('*.pkl'))
# print([post_calibration_routing_fp.stem for post_calibration_routing_fp in post_calibration_routing_fps])
# #export modeled_results_sp so the next step can compute objective functions

# for post_calibration_routing_fp in tqdm(post_calibration_routing_fps):
    
#     # #skip if this has already been performed?
#     # if (config['calibration_fp']/'post_calibration_loss'/(post_calibration_routing_fp.stem+'.pkl')).exists():
#     #     continue
    
#     with post_calibration_routing_fp.open('rb') as fh:
#         results_dict = pickle.load(fh)
    
#     modeled_results_dict = {}
#     # we could try storing these
#     for tripid, item in full_set.items(): # make sure you import this
#         chosen = item['matched_edges'].values
#         shortest = item['shortest_edges'].values
#         od = (item['origin_node'],item['destination_node'])
#         modeled = results_dict[od]['edge_list']
    
#         #compute the loss values (store the intermediates too so total vs mean methods can be compared)
#         modeled_jaccard_exact_intersection, modeled_jaccard_exact_union =  stochastic_optimization.jaccard_exact(chosen,modeled,length_dict)
#         modeled_jaccard_exact = modeled_jaccard_exact_intersection / modeled_jaccard_exact_union
#         modeled_jaccard_buffer_intersection, modeled_jaccard_buffer_union =  stochastic_optimization.jaccard_buffer(chosen,modeled,geo_dict)
#         modeled_jaccard_buffer = modeled_jaccard_buffer_intersection / modeled_jaccard_buffer_union

#         modeled_results_dict[tripid] = {
#             'modeled_edges': pd.DataFrame(modeled,columns=['linkid','reverse_link']),
#             'modeled_length': round(np.array([length_dict.get(tripid[0],0) for tripid in modeled]).sum()/5280,1),
#             'modeled_detour': round(stochastic_optimization.detour_factor(modeled,shortest,length_dict),2),
#             'modeled_jaccard_exact': round(modeled_jaccard_exact,2),
#             'modeled_jaccard_exact_intersection': round(modeled_jaccard_exact_intersection,2),
#             'modeled_jaccard_exact_union': round(modeled_jaccard_exact_union,2),
#             'modeled_jaccard_buffer': round(modeled_jaccard_buffer,2),
#             'modeled_jaccard_buffer_intersection': round(modeled_jaccard_buffer_intersection,2),
#             'modeled_jaccard_buffer_union': round(modeled_jaccard_buffer_union,2),
#         }
    
#     # print('Mean Jaccard Index',round(jaccard_mean,2),'Mean Buffer',round(buffer_mean,2))
#     with (config['calibration_fp']/'post_calibration_loss'/(post_calibration_routing_fp.stem+'.pkl')).open('wb') as fh:
#         pickle.dump(modeled_results_dict,fh)


Have a function that gathers the loss function value for each trip, and one for aggregating them all

In [ ]:
# post_calibration_loss_fps = list((config['calibration_fp']/"post_calibration_loss").glob('*.pkl'))
# print([post_calibration_loss_fp.stem for post_calibration_loss_fp in post_calibration_loss_fps])
# from collections import defaultdict
# all_loss_stats = defaultdict(dict)
# for post_calibration_loss_fp in tqdm(post_calibration_loss_fps):
#     with post_calibration_loss_fp.open('rb') as fh:
#         loss_dict = pickle.load(fh)
#     #remove the edges
#     for tripid, item in loss_dict.items():
#         #remove certain fields
#         for key in list(item.keys()):
#             if ('_intersection' in key) | ('_union' in key) | (key=='modeled_edges'):
#                 item.pop(key)
#         renamed = {post_calibration_loss_fp.stem+'_'+key.removeprefix('modeled_'):sub_item for key, sub_item in item.items()}
#         all_loss_stats[tripid].update(renamed)
# all_loss_stats = pd.DataFrame.from_dict(all_loss_stats,orient='index')
# all_loss_stats

In [ ]:
# # disaggregated
# aggregated_loss = {}
# for post_calibration_loss_fp in tqdm(post_calibration_loss_fps):
#     with post_calibration_loss_fp.open('rb') as fh:
#         loss_dict = pickle.load(fh)
    
#     jaccard_exact_mean = np.array([item['modeled_jaccard_exact'] for tripid, item in loss_dict.items()]).mean()
#     jaccard_exact_total = np.array([(item['modeled_jaccard_exact_intersection'],item['modeled_jaccard_exact_union']) for tripid, item in loss_dict.items()])
#     jaccard_exact_total = jaccard_exact_total[:,0].sum() / jaccard_exact_total[:,1].sum()

#     jaccard_buffer_mean = np.array([item['modeled_jaccard_buffer'] for tripid, item in loss_dict.items()]).mean()
#     jaccard_buffer_total = np.array([(item['modeled_jaccard_buffer_intersection'],item['modeled_jaccard_buffer_union']) for tripid, item in loss_dict.items()])
#     jaccard_buffer_total = jaccard_buffer_total[:,0].sum() / jaccard_buffer_total[:,1].sum()

#     aggregated_loss[post_calibration_loss_fp.stem] = {
#         'jaccard_exact_mean': jaccard_buffer_mean,
#         'jaccard_exact_total': jaccard_exact_total,
#         'jaccard_buffer_mean': jaccard_buffer_mean,
#         'jaccard_buffer_total': jaccard_buffer_total, 
#     }
# aggregated_loss = pd.DataFrame.from_dict(aggregated_loss,orient='index').round(2)
# aggregated_loss

In [ ]:
pd.concat([loss_df,betas_df],axis=1)

In [ ]:
# get fit stats
# TODO get the length weighted versions
shortest_jaccard_mean = np.array([item['shortest_jaccard'] for tripid, item in full_set.items()]).mean()
shortest_buffer_mean = np.array([item['shortest_buffer'] for tripid, item in full_set.items()]).mean()
print('Mean Jaccard Index',round(shortest_jaccard_mean,2),'Mean Buffer',round(shortest_buffer_mean,2))

shortest_jaccard = {tripid:item['shortest_jaccard'] for tripid, item in full_set.items()}
shortest_jaccard = pd.Series(shortest_jaccard)
shortest_jaccard.name = 'shortest_jaccard'

In [ ]:

test[post_calibration_loss_fp.stem] = {
    'modeled_jaccard_exact_mean': modeled_jaccard_buffer_mean,
    'modeled_jaccard_exact_total': modeled_jaccard_exact_total,
    'modeled_jaccard_buffer_mean': modeled_jaccard_buffer_mean,
    'modeled_jaccard_buffer_total': modeled_jaccard_buffer_total, 
}
# get fit stats
# TODO get the length weighted versions
shortest_jaccard_mean = np.array([item['shortest_jaccard'] for tripid, item in full_set.items()]).mean()
shortest_buffer_mean = np.array([item['shortest_buffer'] for tripid, item in full_set.items()]).mean()
print('Mean Jaccard Index',round(shortest_jaccard_mean,2),'Mean Buffer',round(shortest_buffer_mean,2))

shortest_jaccard = {tripid:item['shortest_jaccard'] for tripid, item in full_set.items()}
shortest_jaccard = pd.Series(shortest_jaccard)
shortest_jaccard.name = 'shortest_jaccard'

In [ ]:
jaccard_mean

In [ ]:
jaccrd_total

In [ ]:
x = np.array([(3,4),(3,4)])
x[:,0].sum() / x[:,1].sum()

In [ ]:
x.shape

In [ ]:
# sum of all of the intersects intersect of chosen and modeled / sum of all of the unions

In [ ]:
post_calibration_result_fps = list((config['calibration_fp']/"post_calibration").glob('*.pkl'))

# pick a specific model
specific_model = 'calibration2_new'
calibration_result_fps = [x for x in calibration_result_fps if specific_model == x.stem.split('(')[0].strip()]
post_calibration_result_fps

In [ ]:
#TODO make this into a function

import pandas as pd
import matplotlib.pyplot as plt
#aggregated results and overlap figures
mean_values = []
for idx, post_calibration_result_fp in enumerate(post_calibration_result_fps):
    print('Calibration result',post_calibration_result_fp.stem)
    with post_calibration_result_fp.open('rb') as fh:
        post_calibration_result = pickle.load(fh)

    jaccard_mean = np.array([item['modeled_jaccard'] for tripid, item in post_calibration_result.items()]).mean()
    buffer_mean = np.array([item['modeled_buffer'] for tripid, item in post_calibration_result.items()]).mean()
    mean_values.append((post_calibration_result_fp.stem,round(jaccard_mean,2),round(buffer_mean,2)))

    print('Mean Jaccard Index',round(jaccard_mean,2),'Mean Buffer',round(buffer_mean,2))
    # with (config['calibration_fp']/'post_calibration'/(calibration_result_fp.stem+'.pkl')).open('wb') as fh:
    #     pickle.dump(modeled_results_dict,fh)

    #get just jaccard values
    modeled_jaccard = {tripid:item['modeled_jaccard'] for tripid, item in post_calibration_result.items()}
    modeled_jaccard = pd.Series(modeled_jaccard)
    modeled_jaccard.name = 'modeled_jaccard'
    df = pd.concat([shortest_jaccard,modeled_jaccard],axis=1,ignore_index=False)

    # make figures
    # Create the histogram
    plt.figure(figsize=(12, 12))
    plt.hist(df['shortest_jaccard'], bins=20, alpha=0.5, label='Shortest Path Overlap', color='grey')
    plt.hist(df['modeled_jaccard'], bins=20, alpha=0.3, label='Calibrated Overlap', color='blue')

    # Adding labels, title, and legend with font size adjustments
    plt.xlabel('Overlap', fontsize=22)
    plt.ylabel(f'Frequency (N={df.shape[0]})', fontsize=22)
    plt.title(post_calibration_result_fp.stem, fontsize=16)
    plt.legend(title='Jaccard Index', fontsize=22, title_fontsize=22)
    plt.ylim([0,700])

    # Adjusting the font size of the tick labels
    plt.xticks(fontsize=22)
    plt.yticks(fontsize=22)

    # Show the plot
    plt.savefig(config['calibration_fp']/'calibration_performance'/(post_calibration_result_fp.stem + '.png'))


In [ ]:
calibration_result['betas_tup']

In [ ]:
metrics = pd.DataFrame(mean_values,columns=['calibration_name','jaccard_mean','buffer_mean'])
metrics.set_index('calibration_name',inplace=True)
metrics.sort_values('jaccard_mean',ascending=False)

In [ ]:
metrics.sort_values('buffer_mean',ascending=False)

In [ ]:
beta_vals

# Calculate all the possible overlap metrics

In [ ]:
import impedance_calibration.src.summarize_route as summarize_route
reload(summarize_route)

chosen_attr = {tripid:summarize_route.route_attributes1(tripid,full_set[tripid]['matched_edges'].values,links,turns) for tripid in full_set.keys()}
shortest_attr = {tripid:summarize_route.route_attributes1(tripid,full_set[tripid]['shortest_edges'].values,links,turns) for tripid in full_set.keys()} 
modeled_attr = {tripid:summarize_route.route_attributes1(tripid,full_set[tripid]['modeled_edges'].values,links,turns) for tripid in full_set.keys()}

In [ ]:
chosen_attr = pd.DataFrame.from_dict(chosen_attr,orient='index')
shortest_attr = pd.DataFrame.from_dict(shortest_attr,orient='index')
modeled_attr = pd.DataFrame.from_dict(modeled_attr,orient='index')

In [ ]:
results_df.reset_index().to_csv(config['calibration_fp']/'objective_functions.csv',index=False)

In [ ]:
results_df = pd.read_csv(config['calibration_fp']/'objective_functions.csv')

In [ ]:
results_df.mean().drop('index').to_dict()

# Segmenting/labeling trips
Want to know which attributes help lead to a loss function value. 

## Tree #1
Want to see if percent detour or if any of the chosen route characterstics (that were accounted for in the calibration process) contributed to the overlap value (modeled_jaccard)

In [ ]:
tree_df = pd.concat([results_df[['chosen_length','chosen_detour','modeled_jaccard']],chosen_attr],axis=1)

tree_df['lanes_above_1'] = tree_df['lane_2'] + tree_df['lane_3']
tree_df.drop(columns=['lane_0','lane_1','lane_2','lane_3'],inplace=True)

First split is the chosen detour rate. 1,377 trips (over half) have a detour rate above 10.5% and this is associated with a bigger reduction in overlap. Past this it appears that trips that took a lot 

Second split is on chosen length and chosen detour rate. 

In [ ]:
from sklearn.tree import plot_tree
from sklearn import tree

X, y = tree_df.drop(columns=['modeled_jaccard']).values, tree_df['modeled_jaccard'].values
clf = tree.DecisionTreeRegressor(max_depth=3,min_samples_split=50)
clf = clf.fit(X, y)
fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
plot_tree(clf, feature_names=tree_df.drop(columns=['modeled_jaccard']).columns, filled=True, ax=ax)
plt.show()

In [ ]:
#normal regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

X, y = tree_df.drop(columns=['modeled_jaccard']), tree_df['modeled_jaccard']
X = sm.add_constant(X)
model_sm = sm.OLS(y, X).fit()
print(model_sm.summary())

In [ ]:
tree_df.sort_values('unsig_major_road_crossing',ascending=False).head(40)

In [ ]:
X, y = one_hot_data.values, merged['impedance'].values
clf = tree.DecisionTreeRegressor(max_depth=5)
clf = clf.fit(X, y)
fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
plot_tree(clf, feature_names=one_hot_data.columns, filled=True, ax=ax)
plt.show()
how to compare predicted values vs actual?
SS_res = ((y - clf.predict(X))**2).sum()
SS_tot = ((y - y.mean())**2).sum()
R2 = 1 - (SS_res/SS_tot)
R2

In [ ]:
# with (config['calibration_fp']/"full_modeled_results.pkl").open('rb') as fh:
#     modeled_results = pickle.load(fh)
## Add Route Attributes
import impedance_calibration.src.summarize_route as summarize_route
cols_to_summarize = {
    'facility_fwd': "category",
    'AADT': ("threshold",[10000]),
    'truck_pct': ("threshold",[5]),
    'speed': "category",
    'lanes': "category",
    'mixed_traffic_no_facil': "boolean",
    'mixed_traffic_w_facil': "boolean"
}
links.set_index(['linkid','reverse_link'],inplace=True)
turns.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],inplace=True)
links.columns
# #unit conversions
#links['length_mi'] = (links['length_ft'] / 5280).round(2)
#links['ascent_ft'] = (links['ascent_m'] * 3.28084).round(0)
#links.drop(columns=['length_ft','ascent_m'],inplace=True)
test_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in full_set.items()]
test_summary = summarize_route.procees_summary_results(test_summary,config['projected_crs_epsg'])
test_summary.drop(columns=['tripid','geometry']).describe()
train_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in train_set.items()]
train_summary = summarize_route.procees_summary_results(train_summary,config['projected_crs_epsg'])
train_summary.drop(columns=['tripid','geometry']).describe()
train_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in train_set.items()]
train_summary = summarize_route.procees_summary_results(train_summary,config['projected_crs_epsg'])
train_summary.drop(columns=['tripid','geometry']).describe()
summary = pd.concat([test_summary,train_summary],ignore_index=True)
# summary.to_file(config['calibration_fp']/"route_attributes.gpkg",layer='modeled')

In [ ]:
results_df

## Calculate the percent change in impedance at the link level for visualization

In [ ]:
links.columns

In [ ]:
impedance_change = links.copy()
impedance_change['imp_prop'] = (impedance_change['link_cost'] - impedance_change['travel_time_min']) / impedance_change['travel_time_min']
impedance_change['imp_prop'] = impedance_change['imp_prop'].round(3)
impedance_change = impedance_change[impedance_change['reverse_link']==False]

In [ ]:
#todo automate the plot generation for when qgis isn't available
impedance_change.plot('imp_prop')

In [ ]:
impedance_change.to_file(config['calibration_fp']/"network_impedance_change.gpkg")

In [ ]:
impedance_change['imp_prop'].describe()

In [ ]:
sorted(impedance_change['imp_prop'].unique())

In [ ]:
(impedance_change['imp_prop']==0).any()

In [ ]:
impedance_change.imp_prop.round(3).value_counts()

## Train results

In [ ]:
# #link_impedance_col = "adj_travel_time_min"
# base_impedance_col = "travel_time_min"
# stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

# #update impedances
# #betas = #past_betas[np.array(past_vals).argmin()]#x.x
# print(betas)
# stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                           stochastic_optimization.link_impedance_function,
#                           base_impedance_col,
#                           stochastic_optimization.turn_impedance_function,
#                           links,turns,turn_G)

# #find shortest path
# train_results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,links,start_node,end_node) for start_node, end_node in train_ods}

# #calulate objective function
# loss_train = loss_function(train_set,train_results_dict,**loss_function_kwargs)
# loss_train[:,1].mean()

## Test Results

In [ ]:
# #link_impedance_col = "adj_travel_time_min"
# base_impedance_col = "travel_time_min"
# stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

# #update impedances
# #betas = #past_betas[np.array(past_vals).argmin()]#x.x
# print(betas)
# stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                           stochastic_optimization.link_impedance_function,
#                           base_impedance_col,
#                           stochastic_optimization.turn_impedance_function,
#                           links,turns,turn_G)

# #find shortest path
# test_results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node) for start_node, end_node in test_ods}

# #calulate objective function
# loss_test = loss_function(test_set,test_results_dict,**loss_function_kwargs)
# loss_test[:,1].mean()


In [ ]:
# test0 = pd.DataFrame(loss_shortest_test,columns=['tripid','shortest'])
# test1 = pd.DataFrame(loss_test,columns=['tripid','impedance'])
# testing = pd.concat([test0,test1])

# train0 = pd.DataFrame(loss_shortest_train,columns=['tripid','shortest'])
# train1 = pd.DataFrame(loss_train,columns=['tripid','impedance'])
# training = pd.concat([train0,train1])

In [ ]:
# #make dataframe and export results
# testing = pd.DataFrame({'tripid':list(test_set.keys()),'shortest':loss_shortest_test[:,1],'impedance':loss_test})
# testing.to_csv(config['calibration_fp']/'testing_results.csv',index=False)
# training = pd.DataFrame({'tripid':list(train_set.keys()),'shortest':loss_shortest_train[:,1],'impedance':loss_train})
# training.to_csv(config['calibration_fp']/'training_results.csv',index=False)

# Assemble dataframe of overlap metrics
Dataframe that adds in jaccard index, frechet dist, detour percent, etc into the dataframe

In [ ]:
full_set[71].keys()

In [ ]:
full_set[71]

In [ ]:
from importlib import reload
reload(stochastic_optimization)
#calulate objective function
test = stochastic_optimization.general_objective_function(
    stochastic_optimization.frechet_distance,
    full_set,
    results_dict,
    links)
test#print(loss_full[:,1].mean().round(5))

In [ ]:
full = pd.DataFrame({'tripid':list(full_set.keys()),'shortest':loss_shortest_full[:,1],'impedance':loss_full[:,1]})


In [ ]:
full.to_csv(config['calibration_fp']/'training_results.csv',index=False)

# Plot distribution of overlap values

In [ ]:
import pandas as pd
#loss_data = pd.DataFrame({'loss_shortest_full':loss_shortest_full,'loss_full':loss_full})

import matplotlib.pyplot as plt
# Create the histogram
#ax = loss_full_df.plot.hist(stacked=True, bins=20, figsize=(12, 12), color=['grey', 'lightgrey'])

# Create the histogram
plt.figure(figsize=(12, 12))
plt.hist(full['shortest'], bins=20, alpha=0.5, label='Shortest Path Overlap', color='grey')
plt.hist(full['impedance'], bins=20, alpha=0.3, label='Calibrated Overlap', color='blue')

# Adding labels, title, and legend with font size adjustments
plt.xlabel('Overlap', fontsize=22)
plt.ylabel(f'Frequency (N={full.shape[0]})', fontsize=22)
#plt.title('Histogram of Training Losses', fontsize=16)
#plt.legend(title='Tra Overlap', fontsize=22, title_fontsize=22)

# Adjusting the font size of the tick labels
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)

# Show the plot
plt.show()

In [ ]:
# import pandas as pd
# loss_data = pd.DataFrame({'loss_shortest_train':loss_shortest_train,'loss_train':loss_train})

# import matplotlib.pyplot as plt
# # Create the histogram
# #ax = loss_train_df.plot.hist(stacked=True, bins=20, figsize=(12, 12), color=['grey', 'lightgrey'])

# # Create the histogram
# plt.figure(figsize=(12, 12))
# plt.hist(loss_data['loss_shortest_train'], bins=20, alpha=0.5, label='Shortest Path Overlap', color='grey')
# plt.hist(loss_data['loss_train'], bins=20, alpha=0.3, label='Calibrated Overlap', color='blue')

# # Adding labels, title, and legend with font size adjustments
# plt.xlabel('Overlap', fontsize=22)
# plt.ylabel('Frequency', fontsize=22)
# #plt.title('Histogram of Training Losses', fontsize=16)
# plt.legend(title='Training Overlap', fontsize=22, title_fontsize=22)

# # Adjusting the font size of the tick labels
# plt.xticks(fontsize=22)
# plt.yticks(fontsize=22)

# # Show the plot
# plt.show()

In [ ]:
# loss_data = pd.DataFrame({'loss_shortest_test':loss_shortest_test,'loss_test':loss_test})

# import matplotlib.pyplot as plt
# # Create the histogram
# #ax = loss_train_df.plot.hist(stacked=True, bins=20, figsize=(12, 12), color=['grey', 'lightgrey'])

# # Create the histogram
# plt.figure(figsize=(12, 12))
# plt.hist(loss_data['loss_shortest_test'], bins=20, alpha=0.5, label='Shortest Path Overlap', color='grey')
# plt.hist(loss_data['loss_test'], bins=20, alpha=0.3, label='Calibrated Overlap', color='blue')

# # Adding labels, title, and legend with font size adjustments
# plt.xlabel('Overlap', fontsize=22)
# plt.ylabel('Frequency', fontsize=22)
# #plt.title('Histogram of Training Losses', fontsize=16)
# plt.legend(title='Testing Overlap', fontsize=22, title_fontsize=22)

# # Adjusting the font size of the tick labels
# plt.xticks(fontsize=22)
# plt.yticks(fontsize=22)

# # Show the plot
# plt.show()

# Add modeled geometry to results dict for visualization


In [ ]:
#add a new modeled edges field so that we can calculate the modeled edges entry
for tripid, item in full_set.items():
    od = (item['origin_node'],item['destination_node'])
    modeled_edges = full_results_dict.get(od,0).get('edge_list',0)
    if isinstance(modeled_edges,int):
        print(modeled_edges)
    #turn to dataframe
    modeled_edges = pd.DataFrame(modeled_edges,columns=['linkid','reverse_link'])
    full_set[tripid].update({'modeled_edges':modeled_edges})

In [ ]:
# #add a new modeled edges field so that we can calculate the modeled edges entry
# for tripid, item in test_set.items():
#     od = (item['origin_node'],item['destination_node'])
#     modeled_edges = test_results_dict.get(od,0).get('edge_list',0)
#     if isinstance(modeled_edges,int):
#         print(modeled_edges)
#     #turn to dataframe
#     modeled_edges = pd.DataFrame(modeled_edges,columns=['linkid','reverse_link'])
#     test_set[tripid].update({'modeled_edges':modeled_edges})

In [ ]:
# #add a new modeled edges field so that we can calculate the modeled edges entry
# for tripid, item in test_set.items():
#     od = (item['origin_node'],item['destination_node'])
#     modeled_edges = test_results_dict.get(od,0).get('edge_list',0)
#     if isinstance(modeled_edges,int):
#         print(modeled_edges)
#     #turn to dataframe
#     modeled_edges = pd.DataFrame(modeled_edges,columns=['linkid','reverse_link'])
#     test_set[tripid].update({'modeled_edges':modeled_edges})

In [ ]:
#combine the test and train set dictionaries
modeled_results = {}
# modeled_results.update(train_set)
# modeled_results.update(test_set)
modeled_results.update(full_set)

In [ ]:
with (config['calibration_fp']/"full_modeled_results.pkl").open('wb') as fh:
    pickle.dump(modeled_results,fh)

# QAQC (separate into a separate notebook)

In [ ]:
# with (config['calibration_fp']/"full_modeled_results.pkl").open('rb') as fh:
#     modeled_results = pickle.load(fh)
# ## Add Route Attributes
# import summarize_route
# cols_to_summarize = {
#     'facility_fwd': "category",
#     'AADT': ("threshold",[10000]),
#     'truck_pct': ("threshold",[5]),
#     'here_speed': "category",
#     'lanes': "category",
#     'mixed_traffic_no_facil': "boolean",
#     'mixed_traffic_w_facil': "boolean"
# }
# links.set_index(['linkid','reverse_link'],inplace=True)
# turns.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],inplace=True)
# links.columns
# # #unit conversions
# #links['length_mi'] = (links['length_ft'] / 5280).round(2)
# #links['ascent_ft'] = (links['ascent_m'] * 3.28084).round(0)
# #links.drop(columns=['length_ft','ascent_m'],inplace=True)
# test_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in full_set.items()]
# test_summary = summarize_route.procees_summary_results(test_summary,config['projected_crs_epsg'])
# test_summary.drop(columns=['tripid','geometry']).describe()
# train_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in train_set.items()]
# train_summary = summarize_route.procees_summary_results(train_summary,config['projected_crs_epsg'])
# train_summary.drop(columns=['tripid','geometry']).describe()
# train_summary = [summarize_route.route_attributes(key,item,'modeled_edges',cols_to_summarize,links,turns) for key, item in train_set.items()]
# train_summary = summarize_route.procees_summary_results(train_summary,config['projected_crs_epsg'])
# train_summary.drop(columns=['tripid','geometry']).describe()
# summary = pd.concat([test_summary,train_summary],ignore_index=True)
# summary.to_file(config['calibration_fp']/"route_attributes.gpkg",layer='modeled')
# summary.columns
# summary = pd.concat([test_summary,train_summary],ignore_index=True)
# summary.to_file(config['calibration_fp']/"route_attributes.gpkg",layer='modeled')




# # Regression Trees
# from sklearn import tree

# testing = pd.read_csv(config['calibration_fp']/'testing_results.csv')
# training = pd.read_csv(config['calibration_fp']/'training_results.csv')

# #assume that keys are in the right order?
# loss_df = pd.concat([testing,training],ignore_index=True)
# #import trip and user characteristics
# trips_df = pd.read_pickle(cycleatl_fp/"trips_3.pkl")
# users_df = pd.read_pickle(cycleatl_fp/"users_1.pkl")
# trips_df.reset_index(drop=True,inplace=True)
# #import route attributes
# matched_summary = gpd.read_file(config['calibration_fp']/"route_attributes.gpkg",layer="matched")
# shortest_summary = gpd.read_file(config['calibration_fp']/"route_attributes.gpkg",layer="shortest")
# #consolidate trip types
# trips_df.loc[trips_df['trip_type']=='other','trip_type'] = 'Other'
# trips_df.loc[trips_df['trip_type']=='Work-related','trip_type'] = 'Work-Related'
# trips_df['trip_type'].value_counts()
# #replace userid with just the first one
# def take_first(x):
#     if isinstance(x,list):
#         return x[0]
#     return x
# users_df['userid'] = users_df['userid'].apply(take_first)
# #consolidate trip types
# trips_df.loc[trips_df['trip_type']=='other','trip_type'] = 'Other'
# trips_df.loc[trips_df['trip_type']=='Work-related','trip_type'] = 'Work-Related'
# trips_df['trip_type'].value_counts()




# ## Tree #1
# - First tree is on the non-null variables
# - The dist. between shortest and impedance were similar and so are the trees
# - Shorter trips better explained by impedance/shortest path which makes sense
#     - Use this to split longer trips? and retrain?
# - Shopping is the only significant trip type variable
# - Speed above 9 mph is usually better explained by impedance
# ## Tree #1
# - First tree is on the non-null variables
# - The dist. between shortest and impedance were similar and so are the trees
# - Shorter trips better explained by impedance/shortest path which makes sense
#     - Use this to split longer trips? and retrain?
# - Shopping is the only significant trip type variable
# - Speed above 9 mph is usually better explained by impedance
# nonulls = ['trip_type','length_mi','avg_speed_mph','(0,4]_prop', '(4,8]_prop',
#        '(8,inf]_prop', 'AADT_10000_prop', 'facility_fwd_bike lane_prop',
#        'facility_fwd_cycletrack_prop', 'facility_fwd_multi use path_prop',
#        'facility_fwd_sharrow_prop', 'here_speed_1_prop', 'here_speed_2_prop',
#        'here_speed_3_prop', 'here_speed_4_prop', 'lanes_1_prop',
#        'lanes_2_prop', 'lanes_3_prop', 'left', 'right', 'signalized',
#        'straight', 'truck_pct_5_prop', 'uturn']
# nonulls_tree_df = merged[nonulls]
# from sklearn.tree import plot_tree

# #convert nominal categorical to numeric
# one_hot_data = pd.get_dummies(nonulls_tree_df,drop_first=True)
# one_hot_data.columns
# X, y = one_hot_data.values, merged['impedance'].values
# clf = tree.DecisionTreeRegressor(max_depth=3,min_samples_split=50)
# clf = clf.fit(X, y)
# fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
# plot_tree(clf, feature_names=one_hot_data.columns, filled=True, ax=ax)
# plt.show()
# X, y = one_hot_data.values, merged['impedance'].values
# clf = tree.DecisionTreeRegressor(max_depth=5)
# clf = clf.fit(X, y)
# fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
# plot_tree(clf, feature_names=one_hot_data.columns, filled=True, ax=ax)
# plt.show()
# how to compare predicted values vs actual?
# SS_res = ((y - clf.predict(X))**2).sum()
# SS_tot = ((y - y.mean())**2).sum()
# R2 = 1 - (SS_res/SS_tot)
# R2
# ## Tree #2
# This one takes the previous variables and adds the user characterstics. Sample size is smaller due to null values

# - Looks like trip distance is still the dominant one here, really think i should start there
# - When removing trip distance and avg speed, trip type and age 55+ are the best

# **NOTE:** just noticed that age should not be dummies, need to fix that and try again
# import json
# user_data_definitions = json.load((Path.home()/'Documents/GitHub/cycleatlanta/user_data_definition.json').open('rb'))

# #add the 55+ column
# user_data_definitions['age']['6'] = '55+'
# #income has too many nulls
# tree_cols = ['age','gender','rider_history','rider_type'] + nonulls #,'total_distance_ft','avg_speed_mph','count']#,'count']#[,'cycling_freq'
# tree_df = merged[tree_cols]

# #use to detect null values
# isnull = ((tree_df == -1) | (tree_df == 'NULL'))

# #TODO do cross-sectionals to see which combination results in the most retained entries
# #remove rows with null values
# tree_df = tree_df[(isnull==False).all(axis=1)]

# loss_vals = merged.loc[tree_df.index]
# get_factor = ['age','rider_history','rider_type']
# # just fyi
# # select_max_cols = ['age','income','cycling_freq']
# # #select the min for these (i.e. strong and fearless over interested but...)
# # select_min_cols = ['rider_type','rider_history']

# for col in get_factor:
#     ivd = {v:k for k, v in user_data_definitions[col].items()}
#     tree_df[col] = tree_df[col].map(ivd)
# tree_df
# #this is where i left off
# #convert nominal categorical to numeric
# dummy_cols = ['gender','trip_type']
# one_hot_data = pd.get_dummies(tree_df[dummy_cols],drop_first=True)
# comb = pd.concat([tree_df.drop(columns=dummy_cols),one_hot_data],ignore_index=False,axis=1)
# comb
# X, y = comb.values, loss_vals['impedance'].values
# clf = tree.DecisionTreeRegressor(max_depth=3,min_samples_split=50)
# clf = clf.fit(X, y)
# #tree.plot_tree(clf,feature_names=one_hot_data.columns)
# # Visualize the tree with higher resolution
# plt.figure(figsize=(20, 10), dpi=300)
# from sklearn.tree import plot_tree
# plot_tree(clf, feature_names=comb.columns, filled=True)
# plt.show()
# # how to compare predicted values vs actual?
# SS_res = ((y - clf.predict(X))**2).sum()
# SS_tot = ((y - y.mean())**2).sum()
# R2 = 1 - (SS_res/SS_tot)
# R2
# What we actually want to know, is what variables help increase the overlap? So wouldn't that be more of an application for linear regression?

# clf
# Since the squared error is already pretty high and our histogram tells us, most of these trips are not currently well predicted with the current impedances. The idea behind using regression trees here is can we figure out if attributes help exmplain the bad overlap?

# Or we could just try knocking off 
# #correlation matrix
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Calculate the correlation matrix
# correlation_matrix = one_hot_data.corr()

# # Display the correlation matrix
# print("Correlation Matrix:")
# print(correlation_matrix)

# # Plot the correlation matrix using seaborn heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Correlation Matrix Heatmap')
# plt.show()

# one_hot_data.corr()
# # Distribution of Loss Function
# # Export to get route attributes
# # Using BRouter Results
# To compare across we'll use Frechet distance. Will need to use next time.
# with (config['calibration_fp']/'brouter_links.txt').open('r') as file:
#     my_list = file.readlines()
#     # Remove any extra whitespace or newline characters
#     my_list = [line.strip() for line in my_list]
# len(my_list)
# geojsons = list((config['calibration_fp']/'GeoJSON_Out').glob('*.geojson'))
# both_ods = list(set.union(set(train_ods),set(test_ods)))
# len(both_ods)
# test_ods
# len(geojsons)
# #use the results dict combined with the geo dict one
# results_dict[(68166811, 8789832117)]
# #
# geojson_geos = []
# for geojson in geojsons:
#     geojson_geo = gpd.read_file(geojson).to_crs(config['projected_crs_epsg'])
#     geojson_geo = np.array(geojson_geo.geometry.item().coords)
#     # geojson_geo = [(x, y) for x, y, z in geojson_geo.coords]
#     # geojson_geo = LineString(geojson_geo)
#     geojson_geos.append(geojson_geo)
# frechet_distance = similaritymeasures.frechet_dist(chosen_coords,modeled_coords)
# #import and compare frechet distance across them with geodict?
# list(geojson_geo.coords)